In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.calibration import calibration_curve
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, brier_score_loss
import joblib
import shap

from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import SMOTE

import matplotlib.pyplot as plt

import sys
import xgboost
import lightgbm

from scipy.stats import chi2

# from sklearnex import patch_sklearn, config_context
# patch_sklearn()

from MLstatkit.stats import Delong_test

import warnings
warnings.filterwarnings('ignore')  # 忽略UserWarning

# data process

In [ ]:
# data = pd.read_csv("data.csv")
data = pd.read_csv("depression_test_process.csv")
data['Cardiac_diseases'] = data[['Diabetes', 'heartdisease', 'stroke']].apply(lambda x: (x==1).sum(), axis=1)
data = data[data['Cardiac_diseases']>=1]
# data['Cardiac_diseases'] = data[['Diabetes', 'Heart disease', 'Stroke']].apply(lambda x: (x==1).sum(), axis=1)
# data = data[data['Cardiac_diseases']>=1]
# data = data.drop(columns=['depression1', 'depression2', 'depression_score', 'Diabetes', 'Heart disease', 'Stroke', 'Cardiac_diseases', 'Hypertension', 'Dyslipidemia', 'Health satisfaction', 'Medical satisfaction']+[c for c in data.columns if 'medicine' in c])
data.replace(to_replace=['', ' '], value=np.nan, inplace=True)
data.drop(columns=['Cardiac_diseases'], inplace=True)
data = data.mask(data < 0)
data

In [ ]:
# data = pd.read_csv("data.csv")
test_data = pd.read_csv("test_data_all_process.csv")
test_data['Cardiac_diseases'] = test_data[['Diabetes', 'heartdisease', 'stroke']].apply(lambda x: (x==1).sum(), axis=1)
test_data = test_data[test_data['Cardiac_diseases']>=1]
# data['Cardiac_diseases'] = data[['Diabetes', 'Heart disease', 'Stroke']].apply(lambda x: (x==1).sum(), axis=1)
# data = data[data['Cardiac_diseases']>=1]
# data = data.drop(columns=['depression1', 'depression2', 'depression_score', 'Diabetes', 'Heart disease', 'Stroke', 'Cardiac_diseases', 'Hypertension', 'Dyslipidemia', 'Health satisfaction', 'Medical satisfaction']+[c for c in data.columns if 'medicine' in c])
test_data.replace(to_replace=['', ' '], value=np.nan, inplace=True)
test_data

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
# 删除缺失值太多的字段
# data_ = data.dropna(axis=1, thresh=data.shape[0]*0.7)
# test_data_ = test_data.dropna(axis=1, thresh=test_data.shape[0]*0.7)
data_ = data.dropna(axis=0, thresh=data.shape[1]*0.7)
test_data_ = test_data.dropna(axis=0, thresh=test_data.shape[1]*0.7)
# data_ = data_.drop(['Unnamed: 0', 'ID'], axis=1)
data_ = data_.drop(columns=['Diabetes', 'heartdisease', 'stroke'])
test_data_ = test_data_.drop(columns=['Diabetes', 'heartdisease', 'stroke', 'Cardiac_diseases'])
test_data_

In [ ]:
data_.info()

In [ ]:
from sklearn.impute import KNNImputer
# KNN填充
idx = data_.index
columns = data_.columns

knn_imputer = KNNImputer(n_neighbors=50)
# 用 KNN 填充缺失值
analysis_data = pd.DataFrame(knn_imputer.fit_transform(data_), columns=data_.columns)
analysis_data.set_index(idx, inplace=True)
analysis_data

In [ ]:
from sklearn.impute import KNNImputer
# KNN填充
idx = test_data_.index
columns = test_data_.columns

knn_imputer = KNNImputer(n_neighbors=50)
# 用 KNN 填充缺失值
analysis_test_data = pd.DataFrame(knn_imputer.fit_transform(test_data_), columns=test_data_.columns)
analysis_test_data.set_index(idx, inplace=True)
analysis_test_data

In [ ]:
analysis_test_data = analysis_test_data.astype(int)
analysis_data = analysis_data.astype(int)

In [ ]:
analysis_data.info()

# feature engineer

In [ ]:
from sklearn.feature_selection import VarianceThreshold

X = analysis_data.drop(columns=['depression'])
y = analysis_data['depression']

# 去掉变化小的变量
selector = VarianceThreshold(threshold=0.05)  # 可以调整阈值
X_var_thresh = selector.fit_transform(X)

# 获取剩余特征的列名
features_var_thresh = X.columns[selector.get_support()]
X = analysis_data[features_var_thresh]
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

# 测试集转化
test_data_X = analysis_test_data.drop(columns=['depression'])
test_data_y = analysis_test_data['depression']
test_data_X = test_data_X.reset_index(drop=True)
test_data_y = test_data_y.reset_index(drop=True)

X

In [ ]:
# ------------------- lasso --------------------------------
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

# 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# LassoCV 进行特征选择
lasso = LassoCV(cv=10, max_iter=30000)
lasso.fit(X_scaled, y)

# 获取非零系数的特征
# features_lasso = X.columns[lasso.coef_>0].to_list()[:15]
# features_lasso, len(features_lasso)

# 获取所有特征的系数
coef = lasso.coef_

# 将特征名称和对应的系数放入DataFrame
coef_df = pd.DataFrame({'Feature': X.columns, 'Coefficient': coef})

# 按系数的绝对值从大到小排序
coef_df['AbsCoefficient'] = coef_df['Coefficient'].abs()
coef_df = coef_df.sort_values(by='AbsCoefficient', ascending=False)

# 选择前15个特征
features_lasso = coef_df.head(15)['Feature'].to_list()
features_lasso, len(features_lasso)

# 建模

In [ ]:
def calculate_metrics(y_test, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    ppv = tp / (tp + fp)  # precision
    npv = tn / (tn + fn)
    return sensitivity, specificity, ppv, npv

def hosmer_lemeshow_test(y_true, y_pred_prob, g=10):
    data = pd.DataFrame({'true': y_true, 'pred': y_pred_prob})
    data['group'] = pd.qcut(data['pred'], g, duplicates='drop')
    obs = data.groupby('group')['true'].sum()
    exp = data.groupby('group')['pred'].sum()
    n = data.groupby('group').size()
    hl_test_stat = ((obs - exp) ** 2 / (exp * (1 - exp / n))).sum()
    p_value = 1 - chi2.cdf(hl_test_stat, g - 2)
    return hl_test_stat, p_value


In [ ]:
# 校准曲线
def calibration_curve_plot(y_test, y_pred_prod, file, n_bins=10):
    prob_true, prob_pred = calibration_curve(y_test, y_pred_prod, n_bins=10)

    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.plot(prob_pred, prob_true, marker='o', label='Bias-corrected')
    plt.plot([0, 1], [0, 1], linestyle='--', label='Ideal')
    plt.xlabel('Predicted Pr (PCOS=1)')
    plt.ylabel('Actual Probability')
    plt.title('Calibration Curve')
    plt.legend()
    plt.savefig(file, dpi=300, bbox_inches = 'tight')
    plt.show()


# 决策曲线
def calculate_net_benefit_model(thresh_group, y_pred_score, y_label):
    net_benefit_model = np.array([])
    for thresh in thresh_group:
        y_pred_label = y_pred_score > thresh
        tn, fp, fn, tp = confusion_matrix(y_label, y_pred_label).ravel()
        n = len(y_label)
        net_benefit = (tp / n) - (fp / n) * (thresh / (1 - thresh))
        net_benefit_model = np.append(net_benefit_model, net_benefit)
    return net_benefit_model


def calculate_net_benefit_all(thresh_group, y_label):
    net_benefit_all = np.array([])
    tn, fp, fn, tp = confusion_matrix(y_label, y_label).ravel()
    total = tp + tn
    for thresh in thresh_group:
        net_benefit = (tp / total) - (tn / total) * (thresh / (1 - thresh))
        net_benefit_all = np.append(net_benefit_all, net_benefit)
    return net_benefit_all


def plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all):
    #Plot
    ax.plot(thresh_group, net_benefit_model, color = 'crimson', label = 'Model')
    ax.plot(thresh_group, net_benefit_all, color = 'black',label = 'Treat all')
    ax.plot((0, 1), (0, 0), color = 'black', linestyle = ':', label = 'Treat none')

    #Fill，显示出模型较于treat all和treat none好的部分
    y2 = np.maximum(net_benefit_all, 0)
    y1 = np.maximum(net_benefit_model, y2)
    ax.fill_between(thresh_group, y1, y2, color = 'crimson', alpha = 0.2)

    #Figure Configuration， 美化一下细节
    ax.set_xlim(0,1)
    ax.set_ylim(net_benefit_model.min() - 0.15, net_benefit_model.max() + 0.15)#adjustify the y axis limitation
    ax.set_xlabel(
        xlabel = 'Threshold Probability', 
        fontdict= {'fontsize': 15}
        )
    ax.set_ylabel(
        ylabel = 'Net Benefit', 
        fontdict= {'fontsize': 15}
        )
    ax.grid('major')
    ax.spines['right'].set_color((0.8, 0.8, 0.8))
    ax.spines['top'].set_color((0.8, 0.8, 0.8))
    ax.legend(loc = 'upper right')

    return ax

def plot_importance(importance, file, feature_names):
    # 对特征重要性进行排序
    indices = np.argsort(importance)[::-1]
    
    # 绘制横向柱状图
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(importance)), importance[indices], align='center')
    plt.yticks(range(len(importance)), [feature_names[i] for i in indices])
    plt.xlabel('Importance')
    plt.title('Feature Importance')
    plt.gca().invert_yaxis()  # 反转y轴以使重要性最高的特征位于顶部
    plt.savefig(file, dpi=300, bbox_inches='tight')
    plt.show()

def plot_roc(file, roc_data):
    plt.figure()
    for model, roc in roc_data.items():
        plt.plot(roc[0], roc[1], label=f'{model} (AUC = {roc[2]:.2f})')
        
    # 添加对角线
    plt.plot([0, 1], [0, 1], 'k--')
    
    # 添加图例和标签
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve Comparison')
    plt.legend(loc='lower right')

    plt.savefig(file, dpi=300, bbox_inches='tight')
    # 显示图像
    plt.show()
    

# 模型评估
def model_eval(name, test_y, predict_y, pred_y_proba):
    print(classification_report(test_y, predict_y))
    print(confusion_matrix(test_y, predict_y))
    print("Accuracy:", accuracy_score(test_y, predict_y))

    auc = roc_auc_score(test_y, pred_y_proba)
    fpr, tpr, _ = roc_curve(test_y, pred_y_proba)
    print("AUC:", auc)

    sensitivity, specificity, ppv, npv = calculate_metrics(test_y, predict_y)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)
    print("PPV:", ppv)
    print("NPV:", npv)

    # H-L test
    hl_stat, hl_p_value = hosmer_lemeshow_test(test_y, pred_y_proba)
    print("H-L test:", hl_stat)
    print("H-L test p-value:", hl_p_value)

    # brier_score_loss
    brier_score = brier_score_loss(test_y, pred_y_proba)
    print("Brier score:", brier_score)

    # 绘制校准曲线
    cv_file = f"output/plot/{name}_{int(auc*100)}_cv.tiff"
    calibration_curve_plot(test_y, pred_y_proba, cv_file)

    # 绘制决策曲线
    thresh_group = np.arange(0, 1, 0.01)
    net_benefit_model = calculate_net_benefit_model(thresh_group, pred_y_proba, test_y)
    net_benefit_all = calculate_net_benefit_all(thresh_group, test_y)
    fig, ax = plt.subplots()
    ax = plot_DCA(ax, thresh_group, net_benefit_model, net_benefit_all)
    dca_file = f"output/plot/{name}_{int(auc*100)}_dca.tiff"
    fig.savefig(dca_file, dpi=300)
    plt.show()

    return auc, fpr, tpr


class DelongTest():
    def __init__(self,preds1,preds2,label,threshold=0.05):
        '''
        preds1:the output of model1
        preds2:the output of model2
        label :the actual label
        '''
        self._preds1=preds1
        self._preds2=preds2
        self._label=label
        self.threshold=threshold
        self._show_result()

    def _auc(self,X, Y)->float:
        return 1/(len(X)*len(Y)) * sum([self._kernel(x, y) for x in X for y in Y])

    def _kernel(self,X, Y)->float:
        '''
        Mann-Whitney statistic
        '''
        return .5 if Y==X else int(Y < X)

    def _structural_components(self,X, Y)->list:
        V10 = [1/len(Y) * sum([self._kernel(x, y) for y in Y]) for x in X]
        V01 = [1/len(X) * sum([self._kernel(x, y) for x in X]) for y in Y]
        return V10, V01

    def _get_S_entry(self,V_A, V_B, auc_A, auc_B)->float:
        return 1/(len(V_A)-1) * sum([(a-auc_A)*(b-auc_B) for a,b in zip(V_A, V_B)])
    
    def _z_score(self,var_A, var_B, covar_AB, auc_A, auc_B):
        return (auc_A - auc_B)/((var_A + var_B - 2*covar_AB )**(.5)+ 1e-8)

    def _group_preds_by_label(self,preds, actual)->list:
        X = [p for (p, a) in zip(preds, actual) if a]
        Y = [p for (p, a) in zip(preds, actual) if not a]
        return X, Y

    def _compute_z_p(self):
        X_A, Y_A = self._group_preds_by_label(self._preds1, self._label)
        X_B, Y_B = self._group_preds_by_label(self._preds2, self._label)

        V_A10, V_A01 = self._structural_components(X_A, Y_A)
        V_B10, V_B01 = self._structural_components(X_B, Y_B)

        auc_A = self._auc(X_A, Y_A)
        auc_B = self._auc(X_B, Y_B)

        # Compute entries of covariance matrix S (covar_AB = covar_BA)
        var_A = (self._get_S_entry(V_A10, V_A10, auc_A, auc_A) * 1/len(V_A10)+ self._get_S_entry(V_A01, V_A01, auc_A, auc_A) * 1/len(V_A01))
        var_B = (self._get_S_entry(V_B10, V_B10, auc_B, auc_B) * 1/len(V_B10)+ self._get_S_entry(V_B01, V_B01, auc_B, auc_B) * 1/len(V_B01))
        covar_AB = (self._get_S_entry(V_A10, V_B10, auc_A, auc_B) * 1/len(V_A10)+ self._get_S_entry(V_A01, V_B01, auc_A, auc_B) * 1/len(V_A01))

        # Two tailed test
        z = self._z_score(var_A, var_B, covar_AB, auc_A, auc_B)
        p = st.norm.sf(abs(z))*2

        return z,p

    def _show_result(self):
        z,p=self._compute_z_p()
        print(f"z score = {z:.5f};\np value = {p:.5f};")
        if p < self.threshold :print("There is a significant difference")
        else:        print("There is NO significant difference")

In [ ]:
def train_test(models, grid_search, data, features):

    best_models = {}

    train_roc = {}
    vaild_roc = {}
    test_roc = {}

    train_output = {}
    vaild_output = {}
    test_output = {}

    for name, model in models.items():
        date = 'model'
        plot = 'plot'

        train_x, train_y, test_x, test_y, test_data_x, test_data_y = data[name]
        
        print(f'############################# {name} ##############################')
        
        # 参数搜索
        param_grid = GridSearchCV(model, grid_search[name], cv=5)
        param_grid.fit(train_x, train_y)
        best_model = param_grid.best_estimator_
        print("Best Parameters:", param_grid.best_params_)

        # 训练集
        predict_train_y = best_model.predict(train_x)
        pred_train_y_proba = best_model.predict_proba(train_x)[:, 1]

        # 验证集
        predict_y = best_model.predict(test_x)
        pred_y_proba = best_model.predict_proba(test_x)[:, 1]

        # 测试集
        predict_test_y = best_model.predict(test_data_x)
        pred_test_y_proba = best_model.predict_proba(test_data_x)[:, 1]

        print("------------------------------------------------------------------------------------------------------")
        print("----------------------------------------------- train ------------------------------------------------")
        print("------------------------------------------------------------------------------------------------------")
        auc_train, fpr_train, tpr_train = model_eval(f"{name}_train", train_y, predict_train_y, pred_train_y_proba)
        train_roc[name] = [fpr_train, tpr_train, auc_train]
        train_output[name] = pred_train_y_proba
        print("------------------------------------------------------------------------------------------------------")
        print("------------------------------------------------ eval ------------------------------------------------")
        print("------------------------------------------------------------------------------------------------------")
        auc, fpr_vaild, tpr_vaild = model_eval(f"{name}_eval", test_y, predict_y, pred_y_proba)
        vaild_roc[name] = [fpr_vaild, tpr_vaild, auc]
        vaild_output[name] = pred_y_proba
        print("------------------------------------------------------------------------------------------------------")
        print("------------------------------------------------ test ------------------------------------------------")
        print("------------------------------------------------------------------------------------------------------")
        test_auc, fpr_test, tpr_test = model_eval(f"{name}_test", test_data_y, predict_test_y, pred_test_y_proba)
        test_roc[name] = [fpr_test, tpr_test, test_auc]
        test_output[name] = pred_test_y_proba

        joblib_file = f"output/model/{name}_{int(auc*100)}.pkl"
        
        if name not in best_models:
            best_models[name] = auc
            joblib.dump(best_model, joblib_file)
        elif auc >= best_models[name]:
            joblib.dump(best_model, joblib_file)
            best_models[name] = auc

        # 特征重要性
        importance_file = f"output/plot/{name}_{int(auc*100)}_importance.tiff"

        if name == "Logistic_Regression" :
            importances = best_model.coef_[0]
        elif name == "Support_Vector_Machine" or name == "Knn":
            continue
        else:
            importances = best_model.feature_importances_
        
        plot_importance(importances, importance_file, features)

        print('####################################################################')

    # 绘制ROC曲线
    roc_file_train = "output/plot/train_roc.tiff"
    plot_roc(roc_file_train, train_roc)

    roc_file_vaild = "output/plot/vaild_roc.tiff"
    plot_roc(roc_file_vaild, vaild_roc)
    
    roc_file_test = "output/plot/test_roc.tiff"
    plot_roc(roc_file_test, test_roc)

    # z score检验
    models_list = list(models.keys())
    print("################################################# train ################################################")
    for i in range(len(models_list) - 1):
        for j in range(i + 1, len(models_list)):
            z_score, p_value = Delong_test(train_y, train_output[models_list[i]], train_output[models_list[j]])
            print(f"Z-Score: {z_score}, P-Value: {p_value} of {models_list[i]} and {models_list[j]}")

    print("################################################# vaild ################################################")
    for i in range(len(models_list) - 1):
        for j in range(i + 1, len(models_list)):
            z_score, p_value = Delong_test(test_y, vaild_output[models_list[i]], vaild_output[models_list[j]])
            print(f"Z-Score: {z_score}, P-Value: {p_value} of {models_list[i]} and {models_list[j]}")

    print("################################################# test ################################################")
    for i in range(len(models_list) - 1):
        for j in range(i + 1, len(models_list)):
            z_score, p_value = Delong_test(test_data_y, test_output[models_list[i]], test_output[models_list[j]])
            print(f"Z-Score: {z_score}, P-Value: {p_value} of {models_list[i]} and {models_list[j]}")
    


def build_and_optimize(fun, features):
    print(f"------------------- {fun} ----------------------")
    # X_train, X_test, y_train, y_test = train_test_split(X[features], y, test_size=0.3, random_state=7909)
    test_data_x = test_data_X[features]
    skf = StratifiedShuffleSplit(n_splits=10)
    for train_index, test_index in skf.split(X[features], y):
        X_train = X.loc[train_index, features]
        X_test = X.loc[test_index, features]
        y_train = y[train_index]
        y_test = y[test_index]

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        test_data_X_scaled = scaler.fit_transform(test_data_x)
    
        # 使用SMOTE进行过采样
        smote = SMOTE(random_state=269)
        X_train_scaled_res, y_train_scaled_res = smote.fit_resample(X_train_scaled, y_train)
    
        X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    
        models = {
            "Logistic_Regression": LogisticRegression(class_weight='balanced'),
            "Support_Vector_Machine": SVC(class_weight='balanced', probability=True),
            "Random_Forest": BalancedRandomForestClassifier(),
            "Gradient_Boosting_Machine": GradientBoostingClassifier(),
            "Xgboost": xgboost.XGBClassifier(tree_method='gpu_hist'),
            "LightGBM": lightgbm.LGBMClassifier(device='gpu',verbose=-1),
            "Knn": KNeighborsClassifier()
        }
    
        grids = {
            "Logistic_Regression": {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2'], 'solver': ['liblinear']},
            "Support_Vector_Machine": {'C': [0.1, 1, 10, 100], 'kernel': ['rbf', 'sigmoid'], 'gamma': ['scale', 'auto']},
            "Random_Forest": {'n_estimators': [50, 100, 300, 500, 600], 'max_depth': [None, 10, 25, 40, 50], 'min_samples_split': [2, 5, 8, 12]},
            "Gradient_Boosting_Machine": {'n_estimators': [50, 100, 300, 500, 600], 'learning_rate': [0.01, 0.1, 0.5], 'max_depth': [3, 5, 8, 12]},
            "Xgboost": {'n_estimators': [50, 100, 300, 500, 600], 'max_depth': [3, 4, 6, 9], 'learning_rate': [0.01, 0.1, 0.5], 'subsample': [0.8, 1.0], 'colsample_bytree': [0.8, 1.0]},
            "LightGBM": {'num_leaves': [5, 15, 31, 50, 70], 'max_depth': [-1, 15, 30], 'learning_rate': [0.01, 0.1, 0.5], 'n_estimators': [100, 300, 500], 'subsample': [0.8, 1.0], 'colsample_bytree': [0.8, 1.0]},
            "Knn": {'n_neighbors': [40, 45, 50, 100, 150]}
        }
    
        data = {
            "Logistic_Regression": [X_train_scaled, y_train, X_test_scaled, y_test, test_data_X_scaled, test_data_y],
            "Support_Vector_Machine": [X_train_scaled, y_train, X_test_scaled, y_test, test_data_X_scaled, test_data_y],
            "Random_Forest": [X_train, y_train, X_test, y_test, test_data_x, test_data_y],
            "Gradient_Boosting_Machine": [X_train, y_train, X_test, y_test, test_data_x, test_data_y],
            "Xgboost": [X_train, y_train, X_test, y_test, test_data_x, test_data_y],
            "LightGBM": [X_train, y_train, X_test, y_test, test_data_x, test_data_y],
            "Knn": [X_train_scaled, y_train, X_test_scaled, y_test, test_data_X_scaled, test_data_y],
        }

        # data = [X_train_scaled, y_train, X_test_scaled, y_test, test_data_X_scaled, test_data_y]
    
        train_test(models, grids, data, features)

In [ ]:
datasets = {"features_lasso": features_lasso}
for key, value in datasets.items():
    build_and_optimize(key, value)